In [14]:
from bs4 import BeautifulSoup
import requests
import json,codecs
import pandas as pd
import re
import os


def request_url(url,method="GET",data=None):
    # เพิ่มส่วนหัว (Headers) เพื่อเลียนแบบเบราว์เซอร์
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
    }
    response = None
    if method == "GET":
        response = requests.get(url, headers=headers)
    else:
        # กำหนด headers ด้วย Content-Type เป็น application/json
        headers["Content-Type"] = "application/json"

        # ส่งคำขอ POST
        response = requests.post(url, data=json.dumps(data), headers=headers)      

    # ตรวจสอบสถานะการตอบกลับ
    if response.status_code == 200:
        return response
    elif response.status_code == 403:
        print("Access is forbidden. Try modifying headers or check if the server blocks scraping.")
    elif response.status_code == 404:
        print("Page not found. Check the URL.")
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        print(f"Failed to retrieve the webpage. Status code: {response.reason}")

# อ่านไฟล์ JSON
file_path = 'data/flood-index-thairath.json'

with open(file_path, 'r', encoding='utf-8') as file:
    news = json.load(file)  # อ่านข้อมูล JSON และแปลงเป็น object ใน Python

for index, item in enumerate(news):
    directory = 'data-detail'
    filenames = os.listdir(directory)
    
    news_id = item["tags"][-1]
    if f"{news_id}.json" in filenames : 
        print(f"SKIP : {news_id}")
        continue

    # Step 1: ระบุ URL ของหน้าเว็บ
    url = item["url"]
    # url = "https://www.thairath.co.th/news/local/south/2831171"
    # url = "https://cs.vru.ac.th/"

    # Step 2: ใช้ requests เพื่อดึง HTML
    response = request_url(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # print(soup.prettify())
    div1 = soup.select("div.__item_article-headline")[0]
    div2 = soup.select("div.__item_article-content")[0]

    title = div1.text
    content =  div2.text
    print(len(title),len(content))
    print(title)
    print(content)

    # Step 3: ใช้ requests เพื่อดึง NER
    # URL ของ API
    url = "https://ml.ckartisan.com/ner/text"
    # ข้อมูลที่ต้องการส่งในรูปแบบ JSON
    data = {
        "content": f"{title} {content}",
    }
    response = request_url(url,"POST",data)    
    ner = response.json()
    # data["data_length"] = len(data["original_message"])
    print(ner)
    item["ner_obj"] = ner

    news_id = item["tags"][-1]
    filename = f"data-detail/{news_id}.json"
    with codecs.open(filename, 'w', encoding='utf-8') as f:
        json.dump(item, f, ensure_ascii=False)
        print(f"Write file {filename}")
    
    print(f"DONE: {index+1}/{len(news)} : {round(index+1/len(news)*100,0)}%")

    # exit from Loop
    # break



SKIP : 2833704
SKIP : 2831296
SKIP : 2831171
SKIP : 2831027
55 4090
น้ำท่วมใต้ ทางขาดทั้งสายเอเชีย-รถไฟ น้ำป่าซัดดินถล่มซ้ำ
ใต้วิกฤติชุมพรฝนหนักต่อเนื่อง ถนนเอเชีย 41 เส้นทางหลักลงใต้น้ำท่วมสูงหลายจุด รถทุกชนิด ผ่านไม่ได้ ต้องเลี่ยงไปใช้ถนนเลียบฝั่งทะเลอันดามัน ด้าน จ.ระนอง ระยะทางอ้อม ไกลกว่า 200 กม. ด้านผู้โดยสารรถไฟโกลาหล น้ำท่วมรางสูง 1 เมตร ปิดการเดินรถขึ้น-ล่องใต้ทุกขบวน “สุราษฎร์ฯ-นครศรีฯ” โดนหนักไม่แพ้กัน ฝนถล่มทั้งวันทั้งคืน น้ำป่าหลาก ดินสไลด์ โคลนถล่ม หลายพื้นที่จมบาดาลสถานการณ์ฝนตกหนักครอบคลุมทั้ง 8 อำเภอ จ.ชุมพร ตั้งแต่ช่วงค่ำวันที่ 13 ธ.ค.จนถึงเช้าวันที่ 14 ธ.ค. ส่งผลให้เกิดน้ำป่าไหลหลากเข้าท่วมพื้นที่การเกษตร ที่อยู่อาศัย สาธารณูปโภค ถนนสายหลัก สายรอง ถูกน้ำท่วมสูงสัญจรผ่านไม่ได้ โดยเฉพาะถนนเอเชีย 41 เส้นทางหลักขึ้น-ล่องภาคใต้ ฝั่งอ่าวไทย ตั้งแต่สี่แยกปฐมพร ต.ขุนกระทิง อ.เมืองชุมพร ไปจนถึง อ.สวี อ.ทุ่งตะโก อ.หลังสวน อ.ละแม เขตติดต่อ จ.สุราษฎร์ธานี ระยะทางกว่า 100 กม. มีหลายจุดน้ำท่วมขัง รถทุกชนิดผ่านไม่ได้ และทางเข้าวัดพระธาตุสวี ต.สวี ก่อนขึ้นสะพานข้ามแม่น้ำสวี ระดับน้ำ

AttributeError: 'NoneType' object has no attribute 'json'

In [8]:
text = "สวัสดีครับ ผมชื่อจอห์น วันนี้จะพาทุกคนไปเที่ยวที่ทะเล สวยมากๆ เลยครับ"

text[-1]


'บ'